In [1]:
from extract import open_server_users
from extract import open_server_engage
from extract import open_server_payment
from extract import open_server_battle
from extract import open_server_resource
from extract import open_server_troop
from extract import server_open_nd
from extract import open_server_goldfarmer

import numpy as np
from config import ETLConfig

### 1. Open server settings

1. Define the game you want to analyze
2. Define days after server open you want data from
3. Define server_ids

In [4]:
from camel_utils_x.camel_sql import SQLBase

In [5]:
game = 'aoz'

days = 10

server_ids = [123,234,5,6,7,8,9]

### 2. Define server open
Fields in returned dataframe will be used in making queries to each server

In [6]:
# unshifted version of server open
server_open_nd()

,open_time,open_date,open_10_date,open_10_date_id,open_date_id
server_id,,,,,
0,2020-08-10,2020-08-10,2020-08-20,20200820,20200810
1,2018-09-20,2018-09-20,2018-09-30,20180930,20180920
2,2018-10-25,2018-10-25,2018-11-04,20181104,20181025
3,2018-11-16,2018-11-16,2018-11-26,20181126,20181116
4,2018-11-18,2018-11-18,2018-11-28,20181128,20181118
...,...,...,...,...,...
350,2020-11-22,2020-11-22,2020-12-02,20201202,20201122
351,2020-11-23,2020-11-23,2020-12-03,20201203,20201123
352,2020-11-26,2020-11-26,2020-12-06,20201206,20201126


In [7]:
# shifted version of sever open
server_open_nd(offset = 10, days = 20)

,open_time,open_date,open_20_date,open_20_date_id,open_date_id
server_id,,,,,
0,2020-08-20,2020-08-20,2020-09-09,20200909,20200820
1,2018-09-30,2018-09-30,2018-10-20,20181020,20180930
2,2018-11-04,2018-11-04,2018-11-24,20181124,20181104
3,2018-11-26,2018-11-26,2018-12-16,20181216,20181126
4,2018-11-28,2018-11-28,2018-12-18,20181218,20181128
...,...,...,...,...,...
350,2020-12-02,2020-12-02,2020-12-22,20201222,20201202
351,2020-12-03,2020-12-03,2020-12-23,20201223,20201203
352,2020-12-06,2020-12-06,2020-12-26,20201226,20201206


### Making queries

*On User*

In [8]:
user = open_server_users(server_ids = [234],days = 5, offset = 3, fields = ['id','create_time','language'])

UserType: 234: 100%|████████████████████████████████████████| 1/1 [00:03<00:00,  3.31s/it]


**From this line, we see that server 234 is opened on 2020-5-3**

In [9]:
server_open_nd(game).loc[234]

open_time          2020-05-03 00:00:00
open_date                   2020-05-03
open_10_date                2020-05-13
open_10_date_id               20200513
open_date_id                  20200503
Name: 234, dtype: object

**Since we shifted 3 days, the start date wound be 2020-5-6**         
**Setting days = 5, enables stopping at 2020-5-11 (excluded)**

In [10]:
server_open_nd(game, offset = 3, days = 5).loc[234]

open_time         2020-05-06 00:00:00
open_date                  2020-05-06
open_5_date                2020-05-11
open_5_date_id               20200511
open_date_id                 20200506
Name: 234, dtype: object

**Check dates in dataframe**

In [11]:
sorted(user.date.dt.date.unique())

[datetime.date(2020, 5, 6),
 datetime.date(2020, 5, 7),
 datetime.date(2020, 5, 8),
 datetime.date(2020, 5, 9),
 datetime.date(2020, 5, 10)]

*On engage / with same days and offset configurations*

In [12]:
engage = open_server_engage(server_ids = [234], days = 5, offset = 3)

Engagement 234: 100%|███████████████████████████████████████| 1/1 [00:01<00:00,  1.10s/it]


In [13]:
sorted(engage.date.unique())

[datetime.date(2020, 5, 6),
 datetime.date(2020, 5, 7),
 datetime.date(2020, 5, 8),
 datetime.date(2020, 5, 9),
 datetime.date(2020, 5, 10)]

**If a server open dataframe is  passed into query functions, no additional open server queries execution inside queryies**

In [14]:
server_open = server_open_nd(game = game, offset = 3, days = 5)

user = open_server_users(server_ids = [234], fields = ['id','create_time','language'], 
                         server_open = server_open, days = 5,offset = 3)

engage = open_server_engage(server_ids = [234], server_open = server_open, days = 5, offset = 3)

payment = open_server_payment(server_ids = [234], server_open = server_open, days = 5, offset = 3)

battle = open_server_battle(server_ids = [234], server_open = server_open, days = 5, offset = 3)

resource = open_server_resource(server_ids = [234], server_open = server_open, days = 5, offset = 3)

troop = open_server_troop(server_ids = [234], server_open = server_open, days = 5, offset = 3)

goldfarmer = open_server_goldfarmer(server_ids = [234], server_open = server_open, days = 5, offset = 3)

GoldFarmerRob 234: 100%|████████████████████████████████████| 1/1 [00:04<00:00,  4.32s/it]


*Check user table*

In [15]:
sorted(user.date.dt.date.unique())

[datetime.date(2020, 5, 6),
 datetime.date(2020, 5, 7),
 datetime.date(2020, 5, 8),
 datetime.date(2020, 5, 9),
 datetime.date(2020, 5, 10)]

*Check engage table*

In [16]:
sorted(engage.date.unique())

[datetime.date(2020, 5, 6),
 datetime.date(2020, 5, 7),
 datetime.date(2020, 5, 8),
 datetime.date(2020, 5, 9),
 datetime.date(2020, 5, 10)]

*Check payment table*

In [17]:
payment.date.dt.date.unique().tolist()

[datetime.date(2020, 5, 6),
 datetime.date(2020, 5, 7),
 datetime.date(2020, 5, 8),
 datetime.date(2020, 5, 9),
 datetime.date(2020, 5, 10)]

*Check battle table*

In [18]:
sorted(battle.date.unique())

[datetime.date(2020, 5, 6),
 datetime.date(2020, 5, 7),
 datetime.date(2020, 5, 8),
 datetime.date(2020, 5, 9),
 datetime.date(2020, 5, 10)]

*Check resource table*

In [19]:
sorted(resource.date.unique())

[datetime.date(2020, 5, 6),
 datetime.date(2020, 5, 7),
 datetime.date(2020, 5, 8),
 datetime.date(2020, 5, 9),
 datetime.date(2020, 5, 10)]

*Check Troop table*

In [20]:
sorted(troop.date.unique())

[datetime.date(2020, 5, 6),
 datetime.date(2020, 5, 7),
 datetime.date(2020, 5, 8),
 datetime.date(2020, 5, 9),
 datetime.date(2020, 5, 10)]

*Check GoldFarmerRob table*

In [21]:
sorted(goldfarmer.date.unique())

[datetime.date(2020, 5, 6),
 datetime.date(2020, 5, 7),
 datetime.date(2020, 5, 8),
 datetime.date(2020, 5, 9),
 datetime.date(2020, 5, 10)]

------------
------------

In [26]:
from dbx import Dbtools
import pandas as pd

In [30]:
tool = Dbtools.initialize('all','aoz')
conn = tool.get_connection(**tool.get_conn_info(123,'gs'))

pd.read_sql_query('show columns from gold_farmer_rob',conn).Field.values

array(['id', 'user_id', 'date_id', 'create_time', 'atk_id', 'atk_ip',
       'atk_name', 'atk_device_id', 'atk_online', 'atk_user_lvl',
       'atk_city_lvl', 'atk_alli_id', 'atk_alli_full', 'atk_alli_short',
       'atk_server', 'atk_node', 'atk_survive', 'atk_death', 'atk_wound',
       'atk_kill', 'atk_power', 'def_id', 'def_ip', 'def_name',
       'def_device_id', 'def_online', 'def_user_lvl', 'def_city_lvl',
       'def_alli_id', 'def_alli_full', 'def_alli_short', 'def_server',
       'def_node', 'def_survive', 'def_death', 'def_wound', 'def_kill',
       'def_power', 'food', 'oil', 'rare_earth', 'iron', 'quick_fight',
       'win'], dtype=object)

In [37]:
attacker_info = ['id','atk_id', 'atk_ip','atk_name', 'atk_device_id', 
                 'atk_online', 'atk_user_lvl','atk_city_lvl', 
                 'atk_alli_id', 'atk_alli_full', 'atk_alli_short',
                 'atk_server', 'atk_node', 'atk_survive', 'atk_death',
                 'atk_wound','atk_kill', 'atk_power']

defender_info = ['id','def_id', 'def_ip','def_name','def_device_id', 
                 'def_online', 'def_user_lvl', 'def_city_lvl',
                 'def_alli_id', 'def_alli_full', 'def_alli_short',
                 'def_server','def_node', 'def_survive', 'def_death', 
                 'def_wound', 'def_kill','def_power']

battle_info = ['id','date_id', 'create_time', 'food', 
               'oil', 'rare_earth', 'iron', 'quick_fight','win' ]

fields = set(battle_info + attacker_info + defender_info)

In [38]:
fields

{'atk_alli_full',
 'atk_alli_id',
 'atk_alli_short',
 'atk_city_lvl',
 'atk_death',
 'atk_device_id',
 'atk_id',
 'atk_ip',
 'atk_kill',
 'atk_name',
 'atk_node',
 'atk_online',
 'atk_power',
 'atk_server',
 'atk_survive',
 'atk_user_lvl',
 'atk_wound',
 'create_time',
 'date_id',
 'def_alli_full',
 'def_alli_id',
 'def_alli_short',
 'def_city_lvl',
 'def_death',
 'def_device_id',
 'def_id',
 'def_ip',
 'def_kill',
 'def_name',
 'def_node',
 'def_online',
 'def_power',
 'def_server',
 'def_survive',
 'def_user_lvl',
 'def_wound',
 'food',
 'id',
 'iron',
 'oil',
 'quick_fight',
 'rare_earth',
 'win'}